# Integrating AWS API Gateway with Kafka

## Introduction

We will build a **Kafka REST Proxy integration**, which provides a RESTful interface to a Kafka cluster. This makes it easy to produce and consume messages, view the state of a cluster, or perform administrative actions without using native Kafka protocols or clients.


## Kafka REST Proxy

To get started go to the **API Gateway console** and select one of your previously created APIs. Your API should have a `{proxy+}` resource. To set up an integration click on the `ANY` resource, then on the **Edit integration** button:

<p align="center">
    <img src="images/Edit Integration.png" width="700" height="300"/>
</p>

For **Integration type** select **HTTP**. Make sure to also select the **HTTP proxy integration** toggle. 

<p align="center">
    <img src="images/Integration Details.png" width="700" height="450"/>
</p>

HTTP proxy integration is a simple, yet powerful way of building APIs that allow web applications to access multiple resources on the integrated HTTP endpoint. In HTTP proxy integration, API Gateway simply passes client-submitted method requests to the backend. In turn the backend HTTP endpoint parses the incoming data request to determine the appropriate return responses.

For **HTTP method** select `ANY`.

For the **Endpoint URL**, you will need to enter your Kafka Client Amazon EC2 Instance PublicDNS. You can obtain your EC2 Instance Public DNS by navigating to the EC2 console. Here, select your client EC2 machine and look for Public IPv4 DNS and copy this. The endpoint URL should have the following format: `http://<KafkaClientEC2InstancePublicDNS>:8082/{proxy}`


<p align="center">
    <img src="images/Final Integration Prov.png" width="700" height="500"/>
</p>

> By creating a proxy resource with the **{proxy+}** parameter and the **ANY** method, you can provide your integration with access to all available resources.

## Installing Confluent package for REST proxy on EC2 client

To be able to communicate with Kafka on your EC2 you will need to install the  REST proxy package, run the following commands on your EC2 instance:

In [ ]:
# Download the confluent package containing Kafka REST Proxy
sudo wget https://packages.confluent.io/archive/7.2/confluent-7.2.0.tar.gz
# Unzip the tarball file and extract to the confluent directory
tar -xvzf confluent-7.2.0.tar.gz 

## Deploy API

Make note of the **Invoke URL** after deploying the API. Your external Kafka REST Proxy, which was exposed through API Gateway will look like: 

`https://YourAPIInvokeURL/test/topics/<AllYourTopics>` 

You will use this URL to send messages through API Gateway to your Kafka topic.

## Starting the REST proxy

Before sending messages to the API, we need to start our REST proxy so it can recieve requests from API gateway. To do this, first navigate to the `confluent-7.2.0/bin` folder, and then run the following command:

`./kafka-rest-start`

If everything went well, and your proxy is ready to received requests from the API, you should see a **INFO Server started, listening for requests...** in your EC2 console.

## API Responses in Python

Now we can use the Python requests library to test the API and obtain a response. Below, you have an example code structure for sending messages to an AWS API Gateway API using the **Invoke URL**.

In [ ]:
import requests
import json

example_df = {"index": 1, "name": "Maya", "age": 25, "role": "engineer"}

invoke_url = "https://YourAPIInvokeURL/YourDeploymentStage/topics/YourTopicName"
# To send JSON messages you need to follow this structure
payload = json.dumps({
    "records": [
        {
        # Data should be send as pairs of column_name:value, with different columns separated by commas       
        "value": {"index": example_df["index"], "name": example_df["name"], "age": example_df["age"], "role": example_df["role"]}
        }
    ]
})

# This content type is the content type required by the confluent REST proxy
headers = {'Content-Type': 'application/vnd.kafka.json.v2+json'}
response = requests.request("POST", invoke_url, headers=headers, data=payload)

To see whether the request was successfully processed we can `print(response.status_code)`, which should return a status 200, indicating success. Moreover, you should now be able to consume the messages from the Kafka topic.

## Conclusion
At this point, you should have a good understanding of:

* How to configure an API with Kafka REST Proxy Integration.
* How to send requests to a Kafka REST Proxy API using Python